In [ ]:
# Create a virtual environment and install dependencies
#!python -m venv venv

# Activate the virtual environment
! .\env\Scripts\Activate

In [ ]:
!pip install pandas python-dateutil
!pip install kafka-python


In [1]:
import os, json, pandas as pd
from dateutil import tz
from dateutil import parser as dateparser
from datetime import datetime
import time


#### Data preprocessing method definition

In [2]:
## Util methods

# Create the clean column names
def cleanColumnNames(df):
    df.columns = (df.columns.str.strip().str.lower().str.replace(r"\s+", "_", regex=True))
    return df


def validate_format_date_time_objects(df):
    # Convert the date_time into a date time object
    df['datetime'] = pd.to_datetime(df['read_date'], errors='coerce')

    # localize to America/Chicago, then convert to UTC
    s = pd.to_datetime(df['read_date'], errors='coerce')
    s = (s.dt.tz_localize('America/Chicago', nonexistent='shift_forward', ambiguous='NaT').dt.tz_convert('UTC'))
    df['datetime_utc'] = s
    df['timestamp_ms'] = (s.view('int64') // 10**6).astype('Int64')
    return df


def validate_boolean_data(df):
    # Convert the heavy_vehicle column to numeric, coercing errors to NaN
    # Use pandas nullable integer dtype so NaN/empty values are preserved instead of raising on astype(int)
    df['heavy_vehicle'] = pd.to_numeric(df.get('heavy_vehicle', pd.Series()), errors='coerce').astype('Int64')
    return df


def check_null_values(df, display_count):
    # Show missing value counts based on the column names
    print("\n Missing value summary (exact string '',NaN):")
    missing_report = pd.DataFrame({
        'missing_count_na': df.isna().sum(),
        'empty_count_na': (df == '').sum(),
        'unique_values': df.nunique(dropna=False)
    }).sort_values('missing_count_na', ascending=False)
    display(missing_report.head(display_count))


def drop_missing_values(df):
    # Drop rows where ANY value is missing
    df_main_rows = df.dropna()
    return df_main_rows


# Initial load data preprocessing
def preprocess_data(df):
    # clean the column names
    df_clean_cols = cleanColumnNames(df)

    # Validate date objects
    df_date_processed = validate_format_date_time_objects(df_clean_cols)

    # Validate booloean fields
    df_bool_processed = validate_boolean_data(df_date_processed)

    # Treate missing values
    df_clean_missing = drop_missing_values(df_bool_processed)

    return df_clean_missing


### Kafka producer

In [3]:
import json
from kafka import KafkaProducer
import pandas as pd

def initialise_kafka(bootstrap_server):
    # Initialize Kafka producer
    producer = KafkaProducer(
        bootstrap_servers=bootstrap_server,
        value_serializer=lambda v: json.dumps(v, default=str).encode('utf-8')  # serialize dict to JSON bytes
    )
    return producer


# format and clean data to kafka friendly format
def send_to_kafka(producer, topic, df):
    # Convert each row in DataFrame to JSON and send to Kafka
    for _, row in df.iterrows():
        message = row.to_dict()  # convert row to dict
        producer.send(topic, value=message)

    producer.flush()  # ensure all messages sent
    print('Data sent to Kafka topic successfully')
    return



### Read the data chunk wise and run the file

In [ ]:
file_path = "../dataset/Camera_Traffic_Counts_20251113.csv"
initial_load = 500
batch_size = 500
delay_seconds = 5
offset_track =initial_load

# Kafka
bootstrap_server = 'localhost:9092'
topic = 'raw_sensor_data_ingestion'

# initialise the kafka
producer = initialise_kafka(bootstrap_server)

# Create an iterator for reading the CSV in chunks
chunk_iter = pd.read_csv(file_path, chunksize=batch_size)

# Load the initial 100,000 records in one go (if memory permits)
df_initial = pd.read_csv(file_path, nrows=initial_load)
processed_data = preprocess_data(df_initial)
print(f'Initial load: {len(processed_data)} records')

# Initial data load pass to kafka
send_to_kafka(producer, topic, processed_data)

# Then load 12 records every 5 seconds repeatedly
# try:
#     while True:

#         # Create an iterator for reading the CSV in chunks
#         chunk_iter = pd.read_csv(
#             file_path,
#             chunksize=batch_size,
#             skiprows=range(offset_track + 1, offset_track + batch_size)  # skip first initial data set
#         )
#         print(f'start-index: {offset_track + 1} | end-index: {offset_track + batch_size} ')

#         for chunk in chunk_iter:

#             # Process the chunk here
#             processed_chunk = preprocess_data(chunk)
#             print(f'Loaded batch of {len(processed_chunk)} records')

#             # Repeatedly send data chunks to kafka
#             send_to_kafka(producer, topic, processed_chunk)
        
#         # Update the offset_tracker
#         offset_track = offset_track + batch_size

#         break  # Exit the loop
#         time.sleep(delay_seconds)
# except StopIteration:
#     print('End of file reached')


C:\Users\ASUS\AppData\Local\Temp\ipykernel_42656\2597095664.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['datetime'] = pd.to_datetime(df['read_date'], errors='coerce')
C:\Users\ASUS\AppData\Local\Temp\ipykernel_42656\2597095664.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  s = pd.to_datetime(df['read_date'], errors='coerce')
C:\Users\ASUS\AppData\Local\Temp\ipykernel_42656\2597095664.py:17: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  df['timestamp_ms'] = (s.view('int64') // 10**6).astype('Int64')


Initial load: 500 records
Data sent to Kafka topic successfully


: 